# Getting Financial Data - Pandas Datareader

### Introduction:

This time you will get data from a website.


### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as pdr
%matplotlib inline

### Step 2. Create your time range (start and end variables). The start date should be 01/01/2015 and the end should today (whatever your today is).

In [2]:
import time
from datetime import date
from datetime import datetime

In [4]:
today = date.today()
today

datetime.date(2022, 1, 20)

In [ ]:
pd.date_range('2015/1/1',today)

### Step 3. Get an API key for one of the APIs that are supported by Pandas Datareader, preferably for AlphaVantage.

If you do not have an API key for any of the supported APIs, it is easiest to get one for [AlphaVantage](https://www.alphavantage.co/support/#api-key). (Note that the API key is shown directly after the signup. You do *not* receive it via e-mail.)

(For a full list of the APIs that are supported by Pandas Datareader, [see here](https://pydata.github.io/pandas-datareader/readers/index.html). As the APIs are provided by third parties, this list may change.)

### Step 4. Use Pandas Datarader to read the daily time series for the Apple stock (ticker symbol AAPL) between 01/01/2015 and today, assign it to df_apple and print it.

In [5]:
df_apple = pdr.data.DataReader('AAPL','av-daily',start=datetime(2015,1,1),end = today,api_key='1BZAK8FGSXLW8M4F')

In [5]:
df_apple.head()

,open,high,low,close,volume
2015-01-02,111.39,111.44,107.350,109.33,53204626
2015-01-05,108.29,108.65,105.410,106.25,64285491
2015-01-06,106.54,107.43,104.630,106.26,65797116
2015-01-07,107.20,108.20,106.695,107.75,40105934
2015-01-08,109.23,112.15,108.700,111.89,59364547


### Step 5. Add a new column "stock" to the dataframe and add the ticker symbol

In [6]:
df_apple['stock'] = 'AAPL'

In [7]:
df_apple.head(2)

,open,high,low,close,volume,stock
2015-01-02,111.39,111.44,107.35,109.33,53204626,AAPL
2015-01-05,108.29,108.65,105.41,106.25,64285491,AAPL


### Step 6. Repeat the two previous steps for a few other stocks, always creating a new dataframe: Tesla, IBM and Microsoft. (Ticker symbols TSLA, IBM and MSFT.)

In [7]:
Tesla = pdr.data.DataReader('TSLA','av-daily',start=datetime(2015,1,1),end = today,api_key='1BZAK8FGSXLW8M4F')
Tesla['stock'] = 'TSLA'
IBM = pdr.data.DataReader('IBM','av-daily',start=datetime(2015,1,1),end = today,api_key='1BZAK8FGSXLW8M4F')
IBM['stock'] = 'IBM'
Microsoft = pdr.data.DataReader('MSFT','av-daily',start=datetime(2015,1,1),end = today,api_key='1BZAK8FGSXLW8M4F')
Microsoft['stock']='MSFT'

### Step 7. Combine the four separate dataFrames into one combined dataFrame df that holds the information for all four stocks

In [8]:
data = pd.concat([df_apple,Tesla,IBM,Microsoft])
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7100 entries, 2015-01-02 to 2022-01-19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    7100 non-null   float64
 1   high    7100 non-null   float64
 2   low     7100 non-null   float64
 3   close   7100 non-null   float64
 4   volume  7100 non-null   int64  
 5   stock   7100 non-null   object 
dtypes: float64(4), int64(1), object(1)
memory usage: 388.3+ KB


In [11]:
data.describe(include='O')

,stock
count,7092
unique,4
top,AAPL
freq,1773


### Step 8. Shift the stock column into the index (making it a multi-level index consisting of the ticker symbol and the date).

In [56]:
data

,,open,high,low,close,volume
,stock,,,,,
2015-01-02,AAPL,111.3900,111.4400,107.350,109.33,53204626
2015-01-05,AAPL,108.2900,108.6500,105.410,106.25,64285491
2015-01-06,AAPL,106.5400,107.4300,104.630,106.26,65797116
2015-01-07,AAPL,107.2000,108.2000,106.695,107.75,40105934
2015-01-08,AAPL,109.2300,112.1500,108.700,111.89,59364547
...,...,...,...,...,...,...
2022-01-12,MSFT,319.6700,323.4100,317.080,318.27,34372217
2022-01-13,MSFT,320.4658,320.8800,304.000,304.80,45365979
2022-01-14,MSFT,304.2500,310.8200,303.750,310.20,39846414


In [49]:
vol = data['volume'].unstack('stock')

In [52]:
vol.index.name='Date'

In [53]:
vol.head(2)

stock,AAPL,IBM,MSFT,TSLA
Date,,,,
2015-01-02,53204626,5525341,27913852,4764443
2015-01-05,64285491,4880389,39673865,5368477


### Step 8. Aggregate the data of volume to weekly.
Hint: Be careful to not sum data from the same week of 2015 and other years.

In [27]:
vol.index = vol.index.set_levels([pd.to_datetime(vol.index.levels[0]),vol.index.levels[1]])

In [47]:
vol.groupby([vol.index.get_level_values(level=0).year,vol.index.get_level_values(level=0).week,vol.index.get_level_values(level=1)]).sum().unstack('stock')

E:\python3.6\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  """Entry point for launching an IPython kernel.


stock         AAPL       IBM       MSFT       TSLA
2015 1    53204626   5525341   27913852    4764443
     2   282868187  24440360  158596624   22622034
     3   304226647  23272056  157088136   30799137
     4   198737041  31230797  137352632   16215501
     5   465842684  32927307  437786778   15720217
...            ...       ...        ...        ...
2021 51  359176889  17369625   97516470  104781767
     52  337847299  18454937   84417803   91687786
2022 1   482162362  36013766  174012812  153185108
     2   422454950  32044685  193141391  136822673
     3   183415028   9335293   87203600   47028627

[369 rows x 4 columns]

In [57]:
year = vol.index.year
week = vol.index.month

In [60]:
vol1 = vol.groupby([year,week]).sum()

In [63]:
vol1.index.set_names(['year','week'],inplace=True)

In [64]:
vol1

stock            AAPL        IBM       MSFT       TSLA
year week                                             
2015 1     1304879185  117395861  918738022   90121332
     2     1136535493   77654705  656509827  106717638
     3     1138641910  104899758  824335340  122990032
     4      996135511   93821213  874535095   98945034
     5      954152126   57916093  633072750   89127783
...               ...        ...        ...        ...
2021 9     1797948421   76644238  502931199  390215611
     10    1565079040  144097758  516523658  528934537
     11    1688864233  119252012  507120518  646639435
     12    2443227128  113930079  625743806  510209746
2022 1     1088032340   77393744  454357803  337036408

[85 rows x 4 columns]

### Step 9. Find all the volume traded in the year of 2015

In [49]:
data.reset_index(inplace=True)

In [51]:
data['date'] = pd.to_datetime(data['date'])

In [54]:
data.set_index('date',inplace=True)

In [55]:
data[data.index.year==2015]

,stock,open,high,low,close,volume
date,,,,,,
2015-01-02,AAPL,111.39,111.44,107.3500,109.33,53204626
2015-01-05,AAPL,108.29,108.65,105.4100,106.25,64285491
2015-01-06,AAPL,106.54,107.43,104.6300,106.26,65797116
2015-01-07,AAPL,107.20,108.20,106.6950,107.75,40105934
2015-01-08,AAPL,109.23,112.15,108.7000,111.89,59364547
...,...,...,...,...,...,...
2015-12-24,MSFT,55.86,55.96,55.4300,55.67,9570002
2015-12-28,MSFT,55.35,55.95,54.9800,55.95,22458293
2015-12-29,MSFT,56.29,56.85,56.0600,56.55,27731403


In [70]:
vol.reset_index(inplace=True)

In [71]:
vol

stock,Date,AAPL,IBM,MSFT,TSLA
0,2015-01-02,53204626,5525341,27913852,4764443
1,2015-01-05,64285491,4880389,39673865,5368477
2,2015-01-06,65797116,6145670,36447854,6261936
3,2015-01-07,40105934,4701015,29114061,2968390
4,2015-01-08,59364547,4240585,29645202,3442509
...,...,...,...,...,...
1770,2022-01-12,74805173,5352014,34372217,27913005
1771,2022-01-13,84505760,4868349,45365979,32403264
1772,2022-01-14,80440780,5310305,39846414,24308137
1773,2022-01-18,90500236,5236258,41717982,22158412


In [72]:
vol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1775 non-null   datetime64[ns]
 1   AAPL    1775 non-null   int64         
 2   IBM     1775 non-null   int64         
 3   MSFT    1775 non-null   int64         
 4   TSLA    1775 non-null   int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 69.5 KB


In [74]:
vol.Date.

0      2015-01-02
1      2015-01-05
2      2015-01-06
3      2015-01-07
4      2015-01-08
          ...    
1770   2022-01-12
1771   2022-01-13
1772   2022-01-14
1773   2022-01-18
1774   2022-01-19
Name: Date, Length: 1775, dtype: datetime64[ns]